In [1]:
!pip install cw_wrapper

ERROR: Could not find a version that satisfies the requirement cw_wrapper (from versions: none)
ERROR: No matching distribution found for cw_wrapper

[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import time
import numpy as np
from tqdm.autonotebook import tqdm
from cw_wrapper import CWScope, SS1xTarget, make_random_hex

C:\Users\admin\AppData\Local\Temp\ipykernel_2412\1672292181.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


ModuleNotFoundError: No module named 'cw_wrapper'

In [ ]:
# Connecting ChipWhisperer scope and target
scope = CWScope()
scope.connect(ss_version="1.1")


In [ ]:
scope.print_scope_status()

In [ ]:
#%%   # jupyter block separator\

# Programming target
scope.programming_target("kyber-CW308_STM32F3.hex", "STM32F", reset_target_after_programming=True)

In [ ]:
#%%   # jupyter block separator

# Getting SimpleSerial object and printing commands
dut: SS1xTarget = scope.get_simple_serial_target()
scope.reset_target_via_VCC()   # scope.reset_target_via_UFO_nRST()
time.sleep(1)  # Wait for the target device to be ready

# Key setting
# fixed_key = "2AEF4FBF1020489FFD01F8369D353698"
# dut.ss_write("k", 16, fixed_key, following_ack=True)

In [ ]:
dut.get_simpleserial_commands()

In [ ]:
samples = scope.get_status(verbose=False)["samples"]
samples

In [ ]:
scope.get_status()

In [ ]:
scope.set_scope_detail(samples=600, scale='clkgen_x4', offset=0)

In [ ]:
scope.get_last_trig_cnt()

In [ ]:
total_trace = 100000
samples = 600
traces = np.zeros(shape = (total_trace, samples), dtype = np.float32)

cnt = 0

In [ ]:
f = open("a_b_zeta.txt", 'r')
lines = f.readlines()
f.close()

In [ ]:
for i in range(len(lines)):
    lines[i] = lines[i][:-1].strip()
lines    

In [ ]:
a = []
b = []
zeta = []

for i in range(len(lines)):
    temp = []
    
    for k, one in enumerate(lines[i].split()):
        if k<4:
            temp.append(hex(int(one, 16)))
            if k==3:
                a.append(temp)
                temp = []
        elif k<8:
            temp.append(hex(int(one, 16)))
            if k==7:
                b.append(temp)
                temp = []
        else:
            temp.append(hex(int(one, 16)))
            if k==9:
                zeta.append(temp)
                temp = []      
    

In [ ]:
print(b[0][0])

In [ ]:
print("%02x" % (int(b[0][0], 16)))

In [ ]:
a

In [ ]:
input_key =''

for i in a[0]:
    input_key += '%02x' % (int(i, 16))
input_key

In [ ]:
dut.ss_write('k', 4, input_key, following_ack = False)

In [ ]:
what = dut.ss_read('r', 4, following_ack = True)
print(what)

In [ ]:
dut.ss_write('c', 0, '', following_ack = False)
what = dut.ss_read('r', 4, following_ack = True)
print(what)

In [ ]:
from tqdm.notebook import tqdm

cnt = 0
tqdm_progress = tqdm(range(total_trace))

while cnt < total_trace:
    scope.arm()
    print(b[cnt])
    input_b = ''
    
    # 넘겨 줄때는 input_b와 같은 바이트 형식으로 넘겨줘야 한다.
    # 그러면 ['0x2', '0xb', '0xfe', '0xb2'] 와 같은 형태가
    # 020bfeb2와 같은 형태로 넘어가게 되고
    # 받는 쪽에서는 02 0b fe b2 로 받게 된다.
    for i in b[cnt]:
        input_b += "%02x" % (int(i, 16))
    print(input_b)
    dut.ss_write('p', 4, input_b, following_ack = False)

    c = dut.ss_read('r', 4, following_ack=True)
    print(c)
    
    t = scope.get_waveform()

    if t is None:
        continue

    traces[cnt] = t
    tqdm_progress.update(1)
    cnt += 1
    # # Real time plot (jupyter)
    # ax.clear()
    # ax.plot(t)
    # fig.canvas.draw()
    pass        

In [ ]:
print(scope.get_last_trig_cnt())

In [ ]:
np.save("traces.npy", traces)

In [ ]:
scope.get_last_trig_cnt()

In [ ]:
traces.shape

In [ ]:
import matplotlib.pyplot as  plt
%matplotlib notebook
# plt.figure(figsize=(111))
plt.plot(traces[13])
plt.show()
# ax, fig = plt.subplots(5, 1, figsize=(15,30))

# for i in range(5):
#     fig[i].plot(traces[i])
# plt.show()    

In [ ]:
ax, fig = plt.subplots(5, 1, figsize=(15,30))

for i in range(5):
    fig[i].plot(traces[i])
plt.show()    

In [ ]:
np.mean(traces, axis=1)

In [ ]:
p = make_random_hex(4)
p

In [ ]:
len(p)//2

In [ ]:
#%%   # jupyter block separator

# # Real time plot (jupyter)
# %matplotlib notebook
# import matplotlib.pyplot as plt
# fig = plt.figure()
# ax = fig.add_subplot(111)
# plt.ion()
# fig.show()
# fig.canvas.draw()

# Initializing measurement parameters
total_trace = 500
samples = scope.get_status(verbose=False)["samples"]

# Allocating placeholders
traces = np.empty(shape=(total_trace, samples), dtype=np.float32)
plains, ciphers = [], []

# Measuring power traces
cnt = 0
# tqdm 설정
tqdm_progress = tqdm(range(total_trace))

# cnt는 카운터야
# 우리가 cnt < total_trace --> 500개 까지 트레이스를 제대로 받아오는지를 cnt를 통해서 확인한다.
# cnt가 확인용
while cnt < total_trace:
    # 평문 랜덤하게 설정
    p = make_random_hex(16)
    # 파형 따기 전에 준비신호 무조건 넣어야함
    scope.arm()
    # 평문 넣고 'p' 시리얼 실행
    dut.ss_write('p', 16, p, following_ack=False)
    # 읽어 오는거
    c = dut.ss_read('r', 16, following_ack=True)
    # 파형 수집한거 갖고 오기
    t = scope.get_waveform()
    
    # 가끔식 오류가 뜬다.
    # 파형을 가져 온느 단계에서 의도치 않은 상황이 발생 된다.
    # 근데 그런거는 우리가 몰라.
    # cnt를 업하기 전에 확인을해
    # t 파형을 제대로 갖고 왔는지
    # c 암호문을 제대로 갖고 왔는지
    # 이거 둘 중에 하나라도 못갖고 왔어 -> 비정상
    # 그래서 cnt를 업시킺지 않고 똑가은 거를 다시 따
    if t is None or c is None:
        continue
        
    #
    plains.append(p)
    ciphers.append(c)
    traces[cnt] = t
    tqdm_progress.update(1)
    cnt += 1
    # # Real time plot (jupyter)
    # ax.clear()
    # ax.plot(t)
    # fig.canvas.draw()
    pass

plains = np.array(plains)
ciphers = np.array(ciphers)



In [ ]:
print("trace : ", traces.shape)

import matplotlib.pyplot as plt

plt.plot(traces[0])




In [ ]:
#%%   # jupyter block separator

# Saving power traces and metadata
np.save("./traces.npy", traces)
np.save("./plains.npy", plains)
np.save("./ciphers.npy", ciphers)